In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sys 
import mdtraj

sys.path.insert(0, '/home/wwj/Repo/projects/torchdiffeq/')
sys.path.insert(0, '../')

import torch
from torch.optim import Adam
from torchmd.potentials import ExcludedVolume
from torchmd.sovlers import odeint_adjoint, odeint
from nglview import show_ase, show_file, show_mdtraj
from ase.visualize import view

from nff.nn.layers import GaussianSmearing
from ase import Atoms
from math import sqrt

from torchmd.system import GNNPotentials,PairPotentials,System,Stack
from torchmd.md import Simulations

import ase 


In [ ]:
from ase import Atoms
from ase.constraints import FixBondLengths
from ase.calculators.tip3p import TIP3P, rOH, angleHOH
from ase.md import Langevin
import ase.units as units
from ase.io.trajectory import Trajectory
import numpy as np


# Set up water box at 20 deg C density
x = angleHOH * np.pi / 180 / 2
pos = [[0, 0, 0],
       [0, rOH * np.cos(x), rOH * np.sin(x)],
       [0, rOH * np.cos(x), -rOH * np.sin(x)]]
atoms = Atoms('OH2', positions=pos)

#vol = ((18.01528 / 6.022140857e23) / (0.9982 / 1e24))**(1 / 3.)
#atoms.set_cell((vol, vol, vol))
vol = 10.
atoms.set_cell((vol, vol, vol))
atoms.center()

In [136]:
# visualize molecules 
from nglview import show_ase, show_asetraj

show_ase(atoms)

NGLWidget()

In [137]:
tag = 'tip3p_27mol_equil'
atoms.calc = TIP3P(rc=4.5)

#atoms.constraints = FixBondLengths([(3 * i + j, 3 * i + (j + 1) % 3)
#                                   for i in range(1)
#                                   for j in [0, 1, 2]])

md = Langevin(atoms, 0.1 * units.fs, temperature=50 * units.kB,
              friction=0.01, logfile=tag + '.log')

traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=25)

In [138]:

md.run(4000)

#d.attach(traj.write, interval=2)

In [139]:
h2o_traj = Trajectory(tag + '.traj')
ase.io.write("h2o.pdb", h2o_traj)


In [140]:
h2o_pdb_traj = mdtraj.load_pdb("h2o.pdb")

In [141]:
show_mdtraj(h2o_pdb_traj)

NGLWidget(max_frame=159)

In [18]:
# EMT or LJ
import ase 
from ase import units
from nff.train import get_model
from torchmd.md import NoseHooverChain

uracil = np.load("../data/uracil_dft.npz")
uracil_traj = [Atoms(positions=xyz, numbers=uracil.f.z)  for xyz in uracil.f.R[:2000]]
ase.io.write("uracil.pdb", uracil_traj)
ethanol_pdb_traj = mdtraj.load_pdb("uracil.pdb")

In [11]:
show_mdtraj(ethanol_pdb_traj)

NGLWidget(max_frame=1999)

In [19]:
device = 2

atoms = Atoms(positions=uracil.f.R[0], numbers=uracil.f.z)  

In [20]:
vel_init = (uracil.f.R[1] - uracil.f.R[0])/ ( 0.5 * units.fs )

In [21]:
atoms.set_velocities(vel_init)

system = System(atoms, device=device)

In [22]:
# Initialize FF 

gnn_params = {
    'n_atom_basis': 128,
    'n_filters': 128,
    'n_gaussians': 32,
    'n_convolutions': 3,
    'cutoff': 4.,
    'trainable_gauss': False
}


model = get_model(gnn_params)
GNN = GNNPotentials(model, system.get_batch(), system.get_cell_len(), cutoff=5.0, device=system.device)

# define the equation of motion to propagate 
diffeq = NoseHooverChain(GNN, 
        system,
        Q=50.0, 
        T=298.0 * units.kB,
        num_chains=5, 
        adjoint=True).to(device)

In [23]:
sim = Simulations(system, diffeq)


trajs = sim.simulate(steps=5, frequency=5, dt=0.5 * units.fs)

AttributeError: 'System' object has no attribute 'pdb'

In [195]:
all( atoms.pbc )

False